Probably isn't the cleanest code or makes use of the best coding methods. But I needed raw datasets for a final project and was short on time.

source: https://www.basketball-reference.com

# Read in and Create data frame

In [ ]:
import pandas as pd

In [ ]:
# for loop getting required webpages
years = range(1998,2023)

frames = {}
for y in years:        # NBA season to scrape
  year = y
  # URL to scrape, notice f string:
  frames["url" + str(y)] = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

'''
years = range(1998, 2023)

urls = {f"url{y}": f"https://www.basketball-reference.com/leagues/NBA_{y}_per_game.html" for y in years}
urls
'''

In [ ]:
# contains each link used
frames

In [ ]:
# Messy code :(
# reading in the html pages, adding season column, mvp column.
# pl_frames["pl" + str(season_to)] is just to have each webpage read in linked to it's own unique vairable in the pl_frames dict

season_from = 1997 # for season column
season_to = 98 # for season column
x = 0
y = 0

pl_frames = {}

for i in frames:
  if season_to == 98 or season_to == 99: # for years 1997-98 and 1998-99
    pl_frames["pl" + str(season_from + 1)] = pd.read_html(frames[i])[0]
    pl_frames["pl" + str(season_from + 1)]['Season'] = str(season_from)+'-'+str(season_to)
    pl_frames["pl" + str(season_from + 1)]['MVP'] = False


  elif season_to not in range(10,23): # for years 1999-00 through 2008-09
    season_to = 0 + x
    pl_frames["pl" + str(season_from + 1)] = pd.read_html(frames[i])[0]
    pl_frames["pl" + str(season_from + 1)]['Season'] = str(season_from)+'-0'+str(season_to)
    pl_frames["pl" + str(season_from + 1)]['MVP'] = False
    x += 1

  else: # for years 2009-10 through 2021-22
    season_to = 10 + y
    pl_frames["pl" + str(season_from + 1)] = pd.read_html(frames[i])[0]
    pl_frames["pl" + str(season_from + 1)]['Season'] = str(season_from)+'-'+str(season_to)
    pl_frames["pl" + str(season_from + 1)]['MVP'] = False
    y += 1
  season_from += 1
  season_to += 1


In [ ]:
pl_frames

In [ ]:

# creating the data frame with all tables gathered from webpages
NBA_Player_DF = pd.concat(pl_frames)
NBA_Player_DF = NBA_Player_DF.drop_duplicates()
NBA_Player_DF.drop(NBA_Player_DF[NBA_Player_DF['Rk'] == 'Rk'].index, inplace = True) # don't remember why i added this
NBA_Player_DF.reset_index(drop=True, inplace=True)

'''
# Concatenate all data frames and drop duplicates
nba_player_df = pd.concat(pl_frames).drop_duplicates()

# Drop any rows that have "Rk" as their index value (if present)
nba_player_df.drop(nba_player_df[nba_player_df['Rk'] == 'Rk'].index, inplace=True)

# Reset the index and drop the old index column
nba_player_df.reset_index(drop=True, inplace=True)

# Print the shape of the resulting data frame
print(f"Generated data frame with shape {nba_player_df.shape}")
NBA_Player_DF
'''

In [ ]:
NBA_Player_DF

In [ ]:
# checking for known mvp
NBA_Player_DF[(NBA_Player_DF['Season'] == '2006-07') & (NBA_Player_DF['Player'] == 'Dirk Nowitzki')]


In [ ]:
'''
### If you want to read in webpages and create dataframe without adding new columns

# for loop getting required webpages
years = range(1998,2023)

frames = {}
for y in years:        # NBA season to scrape
  year = y
  # URL to scrape, notice f string:
  frames["url" + str(y)] = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

# read in data
num = 1
pl_frames = {}

for i in frames:
  pl_frames["pl" + str(num)] = pd.read_html(frames[i])[0]
  num += 1

# creating the data frame with all tables gathered from webpages
NBA_Player_DF = pd.concat(pl_frames)
NBA_Player_DF = NBA_Player_DF.drop_duplicates()
NBA_Player_DF.drop(NBA_Player_DF[NBA_Player_DF['Rk'] == 'Rk'].index, inplace = True)
NBA_Player_DF.reset_index(drop=True, inplace=True)

# export data

compression_opts = dict(method='zip',archive_name='NBA_Player_Stats.csv')
NBA_Player_DF.to_csv('NBA_Player_Stats.zip', index=False,compression=compression_opts)
'''

# MVP data

In [ ]:

# read in MVP table for reference and find mvps in NBA_Player_DF
mvp_table = pd.read_html('https://www.basketball-reference.com/awards/mvp.html')[0].droplevel(0, axis = 1)

In [ ]:

# keeping 1997-98 through 2021-22 seasons
season_to_remove = []
start = 1955
end = 56
while end <= 97:
  season_to_remove.append(str(start)+'-'+str(end))
  start += 1
  end += 1

for i in season_to_remove:
  mvp_table.drop(mvp_table[mvp_table['Season'] == i].index, inplace = True)
mvp_table

In [ ]:
# list of players and seasons from mvp table
mvp_season = mvp_table['Season'].to_list()
mvp_player = mvp_table['Player'].to_list()
mvp_player

In [ ]:
mvp_season

In [ ]:

# Gathers index for each MVP into list
mvp_loc = []
for i in range(len(mvp_player)):
  mvp_loc.append(int(NBA_Player_DF[(NBA_Player_DF.Player.str.contains(pat = mvp_player[i])) & (NBA_Player_DF.Season.str.contains(pat = mvp_season[i]))].index[0]))

In [ ]:
mvp_loc

In [ ]:

# sets the MVP column NBA_Player_DF to True if index is in mvp_loc list
for i in mvp_loc:
  NBA_Player_DF.at[i,'MVP'] = True

In [ ]:
# MVP table from nba_player_df
MVPs = NBA_Player_DF.loc[(NBA_Player_DF['MVP'] == True)]
MVPs

In [ ]:
# checking for known mvp like earlier (should be True now)
NBA_Player_DF[(NBA_Player_DF['Season'] == '2006-07') & (NBA_Player_DF['Player'] == 'Dirk Nowitzki')]

In [ ]:

# final data frame
NBA_Player_DF

In [ ]:
# export data

compression_opts = dict(method='zip',archive_name='NBA_Player_Stats.csv')
NBA_Player_DF.to_csv('NBA_Player_Stats.zip', index=False,compression=compression_opts)

In [ ]:
NBA_Player_DF.dtypes

In [ ]:
# smaller table
basic_nba_player_stats = NBA_Player_DF.filter(['Player','Pos','Age','Tm','PTS','AST','TRB','Season'], axis=1)
basic_nba_player_stats